In [2]:
!pip install tensorflow Pillow flask pyngrok matplotlib numpy
NGROK_AUTH_TOKEN = "TOKEN" # KENDİ TOKENİNİZİ GİRİN
if NGROK_AUTH_TOKEN != "SENIN_NGROK_TOKENIN_BURAYA":
    get_ipython().system(f"ngrok authtoken {NGROK_AUTH_TOKEN}")
    print("Ngrok token ayarlandı.")
else:
    print("Lütfen NGROK_AUTH_TOKEN değişkenine geçerli token'ınızı girin veya bu satırı yorumlayıp !ngrok authtoken komutunu manuel çalıştırın.")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Ngrok token ayarlandı.


In [3]:
!mkdir -p templates

In [4]:
%%writefile templates/index.html
<!doctype html>
<html lang="tr">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <title>✨ Sanatsal Stil Transferi ✨</title>
    <style>
        body {
            font-family: 'Roboto', 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 0;
            padding: 20px;
            background-color: #f0f2f5; /* Biraz daha yumuşak bir arka plan */
            color: #333;
            display: flex;
            justify-content: center;
            align-items: flex-start;
            min-height: 100vh;
        }
        .container {
            max-width: 1000px; /* Biraz daha geniş container */
            width: 100%;
            background: white;
            padding: 30px 40px; /* Kenar boşlukları artırıldı */
            border-radius: 16px; /* Daha yuvarlak köşeler */
            box-shadow: 0 10px 30px rgba(0,0,0,0.12);
        }
        header {
            text-align: center;
            margin-bottom: 35px;
            padding-bottom: 25px;
            border-bottom: 1px solid #e0e0e0;
        }
        header h1 {
            color: #1a237e; /* Daha koyu, profesyonel bir mavi */
            margin-bottom: 8px;
            font-size: 2.5em; /* Biraz daha büyük başlık */
            font-weight: 700;
        }
        header p {
            color: #546e7a; /* Biraz daha açık alt başlık rengi */
            font-size: 1.1em;
        }
        .form-section, .results-section, .info-section {
            margin-bottom: 35px;
        }
        label {
            display: block;
            margin-bottom: 10px;
            font-weight: 600;
            color: #37474f;
            font-size: 1em;
        }
        input[type="file"] {
            display: block;
            width: calc(100% - 26px); /* padding ve border için ayar */
            padding: 12px;
            border: 1px solid #bac8d3;
            border-radius: 8px;
            background-color: #f8f9fa;
            margin-bottom: 20px;
            font-size: 0.95em;
            transition: border-color 0.3s ease, box-shadow 0.3s ease;
        }
        input[type="file"]:hover {
            border-color: #8c9eff;
        }
        input[type="file"]:focus {
            border-color: #3d5afe;
            box-shadow: 0 0 0 2px rgba(61, 90, 254, 0.25);
            outline: none;
        }
        input[type="submit"] {
            background-image: linear-gradient(to right, #42a5f5, #1e88e5); /* Gradient buton */
            color: white;
            padding: 14px 30px;
            border: none;
            border-radius: 8px;
            cursor: pointer;
            font-size: 1.1em;
            font-weight: 600;
            transition: all 0.3s ease;
            display: block;
            margin: 25px auto 0 auto;
            box-shadow: 0 4px 15px rgba(0,0,0,0.1);
        }
        input[type="submit"]:hover {
            background-image: linear-gradient(to right, #1e88e5, #1565c0);
            box-shadow: 0 6px 20px rgba(0,0,0,0.15);
            transform: translateY(-2px);
        }
        input[type="submit"]:active {
            transform: translateY(0);
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }

        .results-equation {
            display: flex;
            align-items: center; /* Dikeyde ortala */
            justify-content: space-around; /* Elemanlar arası boşluk */
            margin-top: 20px;
            flex-wrap: wrap; /* Küçük ekranlarda alta sarsın */
        }
        .image-card-equation {
            text-align: center;
            padding: 10px;
            display: flex;
            flex-direction: column;
            align-items: center;
            max-width: 280px; /* Kart genişliği */
            flex-grow: 1; /* Boşluğu doldursun */
        }
        .image-card-equation img {
            max-width: 100%;
            height: auto;
            max-height: 250px; /* Resim yüksekliği limiti */
            border-radius: 12px; /* Daha yuvarlak resim köşeleri */
            border: 2px solid #e0e0e0; /* Belirgin çerçeve */
            box-shadow: 0 5px 15px rgba(0,0,0,0.08);
            margin-bottom: 10px;
            object-fit: contain; /* Resim oranını koru */
            background-color: #fdfdfd; /* Resim yüklenemezse arka plan */
        }
        .image-card-equation p {
            font-weight: 600;
            color: #455a64;
            font-size: 0.95em;
            margin-top: 5px;
        }
        .operator {
            font-size: 3em; /* Operatör boyutu */
            color: #78909c;
            margin: 0 15px; /* Operatör çevresi boşluk */
            align-self: center; /* Dikeyde ortala */
            padding-bottom: 60px; /* Resimlerin altındaki yazılarla hizalamak için */
        }

        .loading, .status-message {
            text-align: center;
            font-size: 1.1em;
            margin-top: 30px;
            padding: 15px;
            border-radius: 8px;
        }
        .loading {
            color: #d35400; /* Daha belirgin yükleme rengi */
            background-color: #fff3e0;
            border: 1px solid #ffe0b2;
            display: none; /* Başlangıçta gizli */
        }
        .error-message {
            color: #c0392b;
            background-color: #fdedec;
            border: 1px solid #f5b7b1;
        }
        .info-section {
            background-color: #f8f9fa; /* Bilgi bölümü için hafif arka plan */
            padding: 20px;
            border-radius: 8px;
            border: 1px solid #e9ecef;
        }
        .info-section h2 {
            color: #1c3faa; /* Bilgi başlığı rengi */
            border-bottom: 2px solid #8c9eff;
            padding-bottom: 10px;
            margin-top: 0;
            margin-bottom: 18px;
            font-size: 1.6em;
        }
        .info-section ul {
            list-style: none; /* Madde işaretlerini kaldır */
            padding-left: 0;
            color: #495057;
        }
        .info-section ul li {
            margin-bottom: 10px;
            padding-left: 25px; /* Madde işareti için yer */
            position: relative;
        }
        .info-section ul li::before {
            content: "🎨"; /* Emoji madde işareti */
            position: absolute;
            left: 0;
            top: 0;
            font-size: 1.1em;
        }
        footer {
            text-align: center;
            margin-top: 50px;
            padding-top: 25px;
            border-top: 1px solid #e0e0e0;
            font-size: 0.9em;
            color: #90a4ae;
        }
        /* Küçük ekranlar için responsive ayarlar */
        @media (max-width: 768px) {
            .results-equation {
                flex-direction: column; /* Operatörleri ve resimleri alt alta getir */
            }
            .operator {
                padding-bottom: 10px; /* Operatör alt boşluğunu azalt */
                margin: 15px 0;
                transform: rotate(90deg); /* Artı ve eşittir işaretlerini dikey yap (isteğe bağlı) */
            }
            .image-card-equation {
                max-width: 80%; /* Kart genişliğini artır */
                margin-bottom:15px;
            }
             header h1 { font-size: 2em; }
        }
        @media (max-width: 480px) {
             header h1 { font-size: 1.8em; }
             header p { font-size: 1em; }
             input[type="submit"] { width: 100%; font-size: 1em; padding: 12px; }
             .image-card-equation img { max-height: 200px; }
        }
    </style>
</head>
<body>
    <div class="container">
        <header>
            <h1>✨ Sanatsal Stil Transferi ✨</h1>
            <p>Yüklediğiniz Resimleri Ünlü Sanat Eserlerinin Stiliyle Birleştirin!</p>
        </header>

        <div class="info-section">
            <h2>Nasıl Çalışır?</h2>
            <ul>
                <li>Bir "içerik" resmi (örneğin bir portre veya manzara) ve bir "stil" resmi (örneğin Van Gogh'un bir tablosu) yükleyin.</li>
                <li>"Resmi Stilize Et" butonuna tıklayarak büyüyü başlatın.</li>
                <li>Akıllı algoritmalarımız, içerik resminizin özünü korurken stil resminin sanatsal dokusunu ve renklerini ona aktaracaktır.</li>
                <li>Sonuç resmi aşağıda görüntülenecektir. Bu işlem, resimlerin karmaşıklığına bağlı olarak birkaç dakika sürebilir.</li>
            </ul>
        </div>

        <div class="form-section">
            <form method="post" enctype="multipart/form-data" onsubmit="showLoading()">
                <div>
                    <label for="content_image">1. İçerik Resminizi Seçin (Ana Resim)</label>
                    <input type="file" name="content_image" id="content_image" accept="image/*" required>
                </div>
                <div>
                    <label for="style_image">2. Stil Resminizi Seçin (Uygulanacak Sanatsal Tarz)</label>
                    <input type="file" name="style_image" id="style_image" accept="image/*" required>
                </div>
                <input type="submit" value="🖼️ Resmi Stilize Et 🚀">
            </form>
        </div>

        <div class="loading" id="loading-message">
            ⏳ Sanat eseri yaratılıyor... Fırçalar çalışıyor, renkler karışıyor! Lütfen bekleyin.
        </div>

        {% if error %}
            <div class="status-message error-message"><strong>Eyvah!</strong> {{ error }}</div>
        {% endif %}

        {% if result_image_filename %}
        <div class="results-section">
            <h2>🎉 İşte Sanatsal Harikanız! 🎉</h2>
            <div class="results-equation">
                <div class="image-card-equation">
                    <img src="{{ url_for('static', filename='uploads/' + content_image_filename) }}" alt="İçerik Resmi">
                    <p>İçerik</p>
                </div>
                <div class="operator">+</div>
                <div class="image-card-equation">
                    <img src="{{ url_for('static', filename='uploads/' + style_image_filename) }}" alt="Stil Resmi">
                    <p>Stil</p>
                </div>
                <div class="operator">=</div>
                <div class="image-card-equation">
                    <img src="{{ url_for('static', filename='results/' + result_image_filename) }}" alt="Stilize Edilmiş Resim">
                    <p>Sonuç</p>
                </div>
            </div>
        </div>
        {% endif %}
        <footer>
            <p>Python, TensorFlow ve Flask ile Geliştirilmiş Stil Transferi Uygulaması</p>
            {% if ngrok_url %}
            <p><small>Canlı Demo Adresi: <a href="{{ ngrok_url }}" target="_blank">{{ ngrok_url }}</a></small></p>
            {% endif %}
        </footer>
    </div>
    <script>
        function showLoading() {
            document.getElementById('loading-message').style.display = 'block';
            const resultsSection = document.querySelector('.results-section');
            if (resultsSection) resultsSection.style.display = 'none';
            const errorMessages = document.querySelectorAll('.status-message.error-message'); // Daha spesifik seçici
            errorMessages.forEach(el => el.style.display = 'none');
        }
        document.addEventListener('DOMContentLoaded', (event) => {
            const resultsSection = document.querySelector('.results-section');
            if (resultsSection && resultsSection.innerHTML.trim() !== '') { // Sadece içerik varsa göster
                 // Eğer `result_image_filename` varsa, resultsSection zaten görünür olmalı,
                 // bu yüzden ek bir `style.display = 'block'` gerekmeyebilir.
                 // Ancak emin olmak için, veya eğer dinamik olarak ekleniyorsa:
                 // resultsSection.style.display = 'flex'; // veya block, layout'a göre
            }
        });
    </script>
</body>
</html>

Writing templates/index.html


In [ ]:
# Adım 3: Flask Uygulaması
# ------------------------------------------------------------------
import os
import tensorflow as tf
import numpy as np
import PIL.Image
import time
import functools
import logging # Daha iyi loglama için

from flask import Flask, request, render_template, send_from_directory
from werkzeug.utils import secure_filename
import uuid
import threading
from pyngrok import ngrok, conf

# --- Uygulama Konfigürasyonları ---
APP_NAME = "StyleTransferApp"
FLASK_PORT = 5000
MAX_IMAGE_DIM = 512 # Resimlerin yeniden boyutlandırılacağı maksimum boyut

# Stil Transferi Parametreleri
CONTENT_LAYERS_CONFIG = ['block5_conv2']
STYLE_LAYERS_CONFIG = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
DEFAULT_EPOCHS = 10             # Daha hızlı testler için 5
DEFAULT_STEPS_PER_EPOCH = 100   # Daha hızlı testler için 50
STYLE_WEIGHT_CONFIG = 1e-2
CONTENT_WEIGHT_CONFIG = 1e4
TOTAL_VARIATION_WEIGHT_CONFIG = 30
OPTIMIZER_LEARNING_RATE = 0.02

# --- Loglama Ayarları ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(APP_NAME)

# --- Yardımcı Fonksiyonlar ---
def tensor_to_image(tensor: tf.Tensor) -> PIL.Image.Image:
    tensor = tensor * 255; tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor) > 3: assert tensor.shape[0] == 1; tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

def load_image_for_styling(path_to_img: str) -> tf.Tensor:
    img = tf.io.read_file(path_to_img); img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    shape = tf.cast(tf.shape(img)[:-1], tf.float32); long_dim = tf.reduce_max(shape)
    scale = MAX_IMAGE_DIM / long_dim; new_shape = tf.cast(shape * scale, tf.int32)
    img = tf.image.resize(img, new_shape); img = img[tf.newaxis, :]
    return img

def clip_0_1(image: tf.Tensor) -> tf.Tensor: return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

def create_vgg_feature_extractor(layer_names: list) -> tf.keras.Model:
    vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet'); vgg.trainable = False
    outputs = [vgg.get_layer(name).output for name in layer_names]; model = tf.keras.Model([vgg.input], outputs)
    return model

def gram_matrix(input_tensor: tf.Tensor) -> tf.Tensor:
    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor); num_locations = tf.cast(input_shape[1] * input_shape[2], tf.float32)
    return result / num_locations

class StyleContentModelExtractor(tf.keras.models.Model):
    def __init__(self, style_layers_list: list, content_layers_list: list):
        super(StyleContentModelExtractor, self).__init__()
        self.feature_extractor = create_vgg_feature_extractor(style_layers_list + content_layers_list)
        self.style_layers = style_layers_list
        self.content_layers = content_layers_list
        self.num_style_layers = len(style_layers_list)
        self.feature_extractor.trainable = False
    def call(self, inputs: tf.Tensor) -> dict:
        inputs_scaled = inputs * 255.0
        preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs_scaled)
        outputs = self.feature_extractor(preprocessed_input)
        style_features, content_features = (outputs[:self.num_style_layers], outputs[self.num_style_layers:])
        style_gram_matrices = [gram_matrix(sf) for sf in style_features]
        content_dict = {name: val for name, val in zip(self.content_layers, content_features)}
        style_dict = {name: val for name, val in zip(self.style_layers, style_gram_matrices)}
        return {'content': content_dict, 'style': style_dict}

# --- Global Model Extractor ---
GLOBAL_EXTRACTOR_MODEL = None # Değişken adını daha açıklayıcı yaptım
ARE_MODELS_INITIALIZED = False

def initialize_global_feature_extractor():
    global GLOBAL_EXTRACTOR_MODEL, ARE_MODELS_INITIALIZED
    if not ARE_MODELS_INITIALIZED:
        logger.info("Global StyleContentModelExtractor yükleniyor...")
        try:
            GLOBAL_EXTRACTOR_MODEL = StyleContentModelExtractor(STYLE_LAYERS_CONFIG, CONTENT_LAYERS_CONFIG)
            # Test çağrısı (opsiyonel)
            # _ = GLOBAL_EXTRACTOR_MODEL(tf.zeros([1, 64, 64, 3])) # Daha küçük boyutla test et
            logger.info("Global StyleContentModelExtractor başarıyla yüklendi.")
            ARE_MODELS_INITIALIZED = True
        except Exception as e:
            logger.error(f"Global model yüklenirken HATA: {e}", exc_info=True)
            ARE_MODELS_INITIALIZED = False
    else:
        logger.info("Global model zaten yüklü.")

# --- Stil Transferi Çekirdek Fonksiyonu ---
def perform_style_transfer_process(content_img_path: str, style_img_path: str, output_img_path: str,
                                   epochs: int = DEFAULT_EPOCHS,
                                   steps_per_epoch: int = DEFAULT_STEPS_PER_EPOCH,
                                   style_weight: float = STYLE_WEIGHT_CONFIG,
                                   content_weight: float = CONTENT_WEIGHT_CONFIG,
                                   total_variation_weight: float = TOTAL_VARIATION_WEIGHT_CONFIG):

    if not ARE_MODELS_INITIALIZED or GLOBAL_EXTRACTOR_MODEL is None:
        logger.error("Stil transferi için global model yüklenemedi.")
        raise RuntimeError("Stil transferi için global model yüklenemedi.")

    logger.info(f"Stil transferi başlatılıyor: İçerik='{os.path.basename(content_img_path)}', Stil='{os.path.basename(style_img_path)}'")

    content_image_tensor = load_image_for_styling(content_img_path)
    style_image_tensor = load_image_for_styling(style_img_path)

    style_targets = GLOBAL_EXTRACTOR_MODEL(style_image_tensor)['style']
    content_targets = GLOBAL_EXTRACTOR_MODEL(content_image_tensor)['content']

    image_to_optimize = tf.Variable(content_image_tensor)
    optimizer = tf.keras.optimizers.Adam(learning_rate=OPTIMIZER_LEARNING_RATE, beta_1=0.99, epsilon=1e-1)

    @tf.function()
    def train_one_style_step(image_var, current_optimizer): # Fonksiyon adını daha spesifik yaptım
        with tf.GradientTape() as tape:
            outputs = GLOBAL_EXTRACTOR_MODEL(image_var)
            current_style_outputs = outputs['style']; current_content_outputs = outputs['content']

            s_loss = tf.add_n([tf.reduce_mean((current_style_outputs[name] - style_targets[name])**2) for name in current_style_outputs.keys()])
            s_loss *= style_weight / len(STYLE_LAYERS_CONFIG)

            c_loss = tf.add_n([tf.reduce_mean((current_content_outputs[name] - content_targets[name])**2) for name in current_content_outputs.keys()])
            c_loss *= content_weight / len(CONTENT_LAYERS_CONFIG)

            total_loss = s_loss + c_loss
            total_loss += total_variation_weight * tf.image.total_variation(image_var)

        gradients = tape.gradient(total_loss, image_var)
        current_optimizer.apply_gradients([(gradients, image_var)])
        image_var.assign(clip_0_1(image_var))
        return total_loss

    logger.info(f"Eğitim başlıyor: {epochs} epoch, {steps_per_epoch} adım/epoch.")
    training_start_time = time.time()
    total_steps_so_far = 0
    for epoch_num in range(epochs):
        epoch_start_time = time.time()
        for step_num in range(steps_per_epoch):
            total_steps_so_far += 1
            loss_value = train_one_style_step(image_to_optimize, optimizer)
            if total_steps_so_far % 50 == 0: # Her 50 adımda bir logla
                try:
                    loss_scalar = float(loss_value.numpy())
                    logger.info(f"Epoch {epoch_num+1}/{epochs}, Adım {step_num+1}/{steps_per_epoch} (Toplam: {total_steps_so_far}), Kayıp: {loss_scalar:.4f}")
                except Exception as log_e:
                    logger.warning(f"Kayıp loglanırken hata: {log_e}. Ham değer: {loss_value.numpy()}")
        logger.info(f"Epoch {epoch_num+1} tamamlandı. Süre: {time.time() - epoch_start_time:.2f}s")

    logger.info(f"Toplam eğitim süresi: {time.time() - training_start_time:.2f}s")

    final_image_pil = tensor_to_image(image_to_optimize)
    final_image_pil.save(output_img_path)
    logger.info(f"Stilize edilmiş resim kaydedildi: {output_img_path}")

# --- Flask Uygulaması ---
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'static/uploads'
app.config['RESULT_FOLDER'] = 'static/results'
app.config['ALLOWED_EXTENSIONS'] = {'png', 'jpg', 'jpeg', 'webp'} # WebP de eklenebilir
NGROK_PUBLIC_URL = None # Ngrok URL'sini saklamak için

logger.info("Ngrok token'ının '!ngrok authtoken' komutu ile ayarlandığı varsayılıyor.")

def is_file_allowed(filename: str) -> bool:
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']

@app.route('/', methods=['GET', 'POST'])
def style_transfer_page(): # Route fonksiyon adını değiştirdim
    error_msg_for_template = None
    global NGROK_PUBLIC_URL

    if not ARE_MODELS_INITIALIZED:
        initialize_global_feature_extractor()
        if not ARE_MODELS_INITIALIZED:
            error_msg_for_template = "Sunucu kritik bir model yükleme hatası nedeniyle hazır değil. Lütfen Colab loglarını kontrol edin."
            logger.critical("MODEL YÜKLENEMEDİ, UYGULAMA KULLANILAMAZ DURUMDA.")
            return render_template('index.html', error=error_msg_for_template, ngrok_url=NGROK_PUBLIC_URL), 503

    if request.method == 'POST':
        if 'content_image' not in request.files or 'style_image' not in request.files:
            error_msg_for_template = "Lütfen hem içerik hem de stil resmini yükleyin."
        elif request.files['content_image'].filename == '' or request.files['style_image'].filename == '':
            error_msg_for_template = "Bir veya daha fazla dosya seçilmedi."
        else:
            content_file = request.files['content_image']
            style_file = request.files['style_image']
            if is_file_allowed(content_file.filename) and is_file_allowed(style_file.filename):
                unique_id = uuid.uuid4().hex[:8]
                content_ext = content_file.filename.rsplit('.',1)[1].lower()
                style_ext = style_file.filename.rsplit('.',1)[1].lower()

                content_fn = f"content_{unique_id}.{content_ext}"
                style_fn = f"style_{unique_id}.{style_ext}"
                result_fn = f"stylized_{unique_id}.png"

                content_p = os.path.join(app.config['UPLOAD_FOLDER'], content_fn)
                style_p = os.path.join(app.config['UPLOAD_FOLDER'], style_fn)
                result_p = os.path.join(app.config['RESULT_FOLDER'], result_fn)

                try:
                    content_file.save(content_p)
                    style_file.save(style_p)
                    logger.info(f"Resimler kaydedildi: İçerik='{content_fn}', Stil='{style_fn}'")

                    # Stil transferini çalıştır
                    perform_style_transfer_process(content_p, style_p, result_p) # epoch ve step varsayılanları kullanır

                    return render_template('index.html',
                                           result_image_filename=result_fn,
                                           content_image_filename=content_fn,
                                           style_image_filename=style_fn,
                                           ngrok_url=NGROK_PUBLIC_URL)
                except Exception as e:
                    logger.error(f"Stil transferi veya dosya işlemi sırasında hata: {e}", exc_info=True)
                    error_msg_for_template = f"Üzgünüz, bir hata oluştu: {str(e)[:100]}..." # Kullanıcıya kısa bir hata göster
            else:
                error_msg_for_template = "Geçersiz dosya türü. Lütfen desteklenen formatlarda (PNG, JPG, JPEG, WebP) resimler yükleyin."

        return render_template('index.html', error=error_msg_for_template, ngrok_url=NGROK_PUBLIC_URL)

    return render_template('index.html', result_image_filename=None, error=error_msg_for_template, ngrok_url=NGROK_PUBLIC_URL)

def start_ngrok_and_flask_app():
    global NGROK_PUBLIC_URL

    try: # Önceki ngrok process'lerini sonlandır
        logger.info("Mevcut ngrok tünelleri/process'leri sonlandırılıyor...")
        ngrok.kill(); time.sleep(1)
    except Exception as e: logger.warning(f"Ngrok sonlandırma sırasında uyarı (göz ardı edilebilir): {e}")

    try:
        logger.info("Yeni ngrok tüneli başlatılıyor...")
        # Token'ın Adım 1'de ayarlandığını varsayıyoruz.
        # conf.get_default().auth_token = "YOUR_TOKEN_IF_NEEDED"
        tunnel = ngrok.connect(FLASK_PORT, name=f"{APP_NAME}_Tunnel") # Tünele isim vermek dashboard'da yardımcı olur
        NGROK_PUBLIC_URL = tunnel.public_url
        logger.info(f"✅ Ngrok Tüneli Aktif: {NGROK_PUBLIC_URL}")
        logger.info(f"👉 Uygulamaya erişmek için bu URL'yi kullanın: {NGROK_PUBLIC_URL}")

        logger.info(f"Flask uygulaması http://localhost:{FLASK_PORT} adresinde başlatılıyor...")
        # Werkzeug loglarını kapatmak için (Python logger kullanıyoruz)
        # flask_log = logging.getLogger('werkzeug')
        # flask_log.setLevel(logging.ERROR)
        app.run(host='0.0.0.0', port=FLASK_PORT, debug=False) # debug=False üretim için daha iyi

    except Exception as e:
        logger.critical(f"Ngrok veya Flask başlatılırken KRİTİK HATA: {e}", exc_info=True)
        logger.critical("Lütfen ngrok token'ınızı kontrol edin ve başka aktif ngrok oturumunuz olmadığından emin olun.")
        logger.critical("Ngrok Dashboard: https://dashboard.ngrok.com/agents")
        NGROK_PUBLIC_URL = None

if __name__ == '__main__':
    for folder in [app.config['UPLOAD_FOLDER'], app.config['RESULT_FOLDER'], "templates"]:
        os.makedirs(folder, exist_ok=True)

    logger.info(f"'{APP_NAME}' uygulaması başlatılıyor...")
    initialize_global_feature_extractor()

    if ARE_MODELS_INITIALIZED:
        start_ngrok_and_flask_app()
    else:
        logger.critical("HATA: Global model yüklenemediği için Flask uygulaması başlatılmıyor.")

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 15:59:34] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 15:59:34] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_22']
Received: inputs=Tensor(shape=(1, 512, 512, 3))
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_22']
Received: inputs=Tensor(shape=(1, 341, 512, 3))
  warnings.warn(msg)
<ipython-input-6-1965550975>:153: DeprecationWarning: Con